# Smart Cities (Winter Semester 22/23)

## Workshop: Simulation Mobility Systems

Welcome to the Workshop!

This notebook is used to collect meaningful data from the simulation you will run during the workshop. The python script is structured in an object oriented way, meaning that dataframes imported from log documents are stored in a 'mosaic' object. If you think you have a good command of the programming language, you can call and manipulate them repeatedly on a new cell of jupyter notebook. 
The data we provide you with will be largely sufficient for this workshop, so you are not expected to manipulate the data again, but rather to observe how parameter changes affect the results.  However, if during your research you notice a data set with a different structure, you can edit or calculate it on this notebook.

For questions, troubleshooting and bugs please reach out to Muzaffer Citir (muzaffer.citir@campus.tu-berlin.de)

#### **Step 1:** Initialize the toolbox and libraries used for Mosaic

In [ ]:
import os
import pandas as pd
from ide3a_toolbox import Mosaic

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#### **Step 2:** Create an object named as *mosaic* and define the path to the Eclipse MOSAIC

In [ ]:
mosaic = Mosaic(mosaic_path=os.path.join(os.getcwd(), "eclipse-mosaic-22.0"))

The method `run_simulation()` will run the selected scenario and display the terminal output in the notebook.

- If you set `visualize=True`, the mosaic visualizer will open in a new tab. 
- Setting `visualize=False` might improve simulation speed.
- If you set a `sim_speed`, the simulation will be slowed down to a desired Real Time Factor. For example, use `sim_speed=1` to execute the simulation in real time or `sim_speed=10` to execute 10 times faster than real time. `sim_speed=None` executes simulations as fast as possible.

#### **Step 3:** Running the simulation

In [ ]:
mosaic.run_simulation(visualize=True, sim_speed=1000)

#### **Step 4:** Import log data saved by Eclipse Mosaic

In [ ]:
result = mosaic.import_data()
#display(result.df_vehicle_updates.head(5))
#display(result.df_vehicle_registration.head(5))

#### **Step 5:** Viewing, Sorting, and Filtering Data

##### **Step 5.1:** Combine two data frames called "df_vehicle_updates" and "df_vehicle_registration" into "cumulative_df"

In [ ]:
cumulative_df = pd.merge(result.df_vehicle_updates, result.df_vehicle_registration[['Name','MappingGroup']], how='left')
cumulative_df = pd.merge(cumulative_df, result.df_vehicle_registration[['Name','MappingVehicleTypeVehicleClass']], how='left')
display(cumulative_df.head(5)) # Change the number of .head(###) how many lines you want to see

##### **Step 5.2:** Filter by Unique Vehicle IDs and Equipment Groups

In [ ]:
all_vehicles = result.df_vehicle_registration.Name.unique().tolist()
adhoc_vehicles = cumulative_df.loc[cumulative_df['MappingGroup']=='AdHoc'].Name.unique().tolist()
cellular_vehicles = cumulative_df.loc[cumulative_df['MappingGroup']=='Cellular'].Name.unique().tolist()
unequipped_vehicles = cumulative_df.loc[cumulative_df['MappingGroup']=='Unequipped'].Name.unique().tolist()

In [ ]:
# Uncomment to see Vehicle ID list by Equipment Group
#display(all_vehicles)
#display(adhoc_vehicles)
#display(cellular_vehicles)
#display(unequipped_vehicles)

##### **Step 5.3:** Create Simulation Statistics

In [ ]:
vehicles_having_routeId_2 = cumulative_df.loc[cumulative_df['RouteId']==2.0].Name.unique().tolist()
cellular_vehicles_having_routeId_2 = cumulative_df.loc[(cumulative_df['RouteId']==2.0) & (cumulative_df['MappingGroup']=='Cellular')].Name.unique().tolist()
adhoc_vehicles_having_routeId_2 = cumulative_df.loc[(cumulative_df['RouteId']==2.0) & (cumulative_df['MappingGroup']=='AdHoc')].Name.unique().tolist()

# Calculate how many vehicles alread got messages from the V2X network
list_of_vehicles_got_message = result.df_navigation_log['Vehicles_List'].values

# Calculate how many vehicles already set the 'targetSpeed=6.94'
list_of_vehicles_past_through_hazardous_zone = result.df_traffic_log[14].loc[result.df_traffic_log[15].str.contains('targetSpeed=6.94')==True]

# Print Info
print(f"Total {len(all_vehicles)} vehicles in the simulation. (AdHoc: {len(adhoc_vehicles)}, Cellular: {len(cellular_vehicles)}, Unequipped: {len(unequipped_vehicles)})")
print(f"\nStatistics from Navigation.log file")
print(f"- Total {len(list_of_vehicles_got_message.tolist())} vehicles got messages from the V2X network due to changing route.")
print(f"- {len(adhoc_vehicles_having_routeId_2)} adhoc and {len(cellular_vehicles_having_routeId_2)} cellular vehicle(s) updated the route!")
print(f"\nStatistics from  Traffic.log file")
print(f"- {len(list_of_vehicles_past_through_hazardous_zone)} vehicle(s) passed through the hazardous zone.")


##### **Step 5.4:** Network Statistics

In [ ]:
adhoc_transmitter = result.df_v2x_message_transmission.loc[result.df_v2x_message_transmission['MessageRoutingDestinationType'] == 'AD_HOC_GEOCAST']
cellular_transmitter = result.df_v2x_message_transmission.loc[result.df_v2x_message_transmission['MessageRoutingDestinationType'] == 'CELL_GEOCAST']
all_receiver = pd.merge(result.df_v2x_message_reception, result.df_vehicle_registration[['Name','MappingGroup']], how='left')
adhoc_receiver = all_receiver.loc[all_receiver['MappingGroup']=='AdHoc']
cellular_receiver = all_receiver.loc[all_receiver['MappingGroup']=='Cellular']

print("\nAdhoc")
print(f"- {adhoc_transmitter.groupby(['Name']).size().sum()} messages transmitted by {adhoc_transmitter.groupby(['Name']).size().count()} Adhoc vehicle(s)")
print(f"- {adhoc_receiver.groupby(['Name']).size().sum()} messages received by {adhoc_receiver.groupby(['Name']).size().count()} Adhoc vehicle(s)")

print("\nCellular")
if cellular_transmitter.loc[cellular_transmitter.Name.str.contains('server')].Name.nunique() > 0:
    print(f"- {cellular_transmitter.groupby(['Name']).size().sum()} messages transmitted by {cellular_transmitter.Name.nunique() - cellular_transmitter.loc[cellular_transmitter.Name.str.contains('server')].Name.nunique()} Cellular vehicle(s)")
    print(f"- {cellular_transmitter.loc[cellular_transmitter.Name.str.contains('server')].Name.nunique()} server(s) and {cellular_receiver.groupby(['Name']).size().sum()} messages received by {cellular_receiver.groupby(['Name']).size().count()} Cellular vehicle(s)")
else:
    print(f"- {cellular_transmitter.groupby(['Name']).size().sum()} messages transmitted by {cellular_transmitter.Name.nunique()} Cellular vehicle(s)")
    print(f"- {cellular_receiver.groupby(['Name']).size().sum()} messages received by {cellular_receiver.groupby(['Name']).size().count()} Cellular vehicle(s)")


##### **Step 5.5:** Auxiliary Statistics

In [ ]:
# Statistics of Vehicles Classes and CO2 Emissions
print(f"\nVehicle Class(es): {cumulative_df['MappingVehicleTypeVehicleClass'].unique()}")
print("Total Emmission: {:.2f} g CO2".format(cumulative_df['VehicleEmissionsCurrentEmissionsCo2'].sum()/1000))
print("Average {:.2f} g CO2 released per vehicle".format(cumulative_df[['Name','VehicleEmissionsAllEmissionsCo2']].groupby(['Name']).max().mean()[0]/1000))

# Plotting Example

In [ ]:
adhoc_receiver.head(100)

In [ ]:
sns.histplot(data=adhoc_receiver, y="Name")

In [ ]:
cellular_receiver.head(20)

In [ ]:
sns.histplot(data=cellular_receiver, y="Name")